In [12]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score
from sklearn.feature_selection import f_classif, chi2
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from imblearn.combine import SMOTEENN, SMOTETomek
from sklearn.utils import shuffle
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler, ClusterCentroids, TomekLinks
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler, RobustScaler, Normalizer, KBinsDiscretizer
from sklearn.feature_selection import VarianceThreshold, SelectKBest, SelectPercentile, GenericUnivariateSelect, RFE
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.decomposition import PCA
from sklearn.naive_bayes import GaussianNB, CategoricalNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.linear_model import LogisticRegression, LinearRegression, SGDClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, VotingClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.exceptions import ConvergenceWarning
from sklearn.preprocessing import OrdinalEncoder

### Load train dataset

In [13]:
salaryClassification = pd.read_csv('data/training.csv', sep = ';')

#treat missing values
salaryClassification = salaryClassification.replace(' ?', np.NaN)
salaryClassification[' workclass'] = salaryClassification[' workclass'].replace(np.NaN, 'Unknown')
salaryClassification[' occupation'] = salaryClassification[' occupation'].replace(np.NaN, 'Other')
salaryClassification[' native-country'] = salaryClassification[' native-country'].replace(np.NaN, 'Other')


#convert 'workclass' from categorical to numeric
oe = OrdinalEncoder()
salaryClassification["workclass"] = oe.fit_transform(salaryClassification[[" workclass"]]).astype(int)
salaryClassification = salaryClassification.drop(' workclass', 1)

#fix column names
salaryClassification['fnlwgt'] = salaryClassification[' fnlwgt']
salaryClassification = salaryClassification.drop(' fnlwgt',1)

#convert 'education' from categorical to numeric
oe = OrdinalEncoder()
salaryClassification["education"] = oe.fit_transform(salaryClassification[[" education"]]).astype(int)
salaryClassification = salaryClassification.drop(' education', 1)

#fix column names
salaryClassification['education-num'] = salaryClassification[' education-num']
salaryClassification = salaryClassification.drop(' education-num',1)

#convert 'education' from categorical to numeric
oe = OrdinalEncoder()
salaryClassification["marital-status"] = oe.fit_transform(salaryClassification[[" marital-status"]]).astype(int)
salaryClassification = salaryClassification.drop(' marital-status', 1)

#convert 'occupation'n' from categorical to numeric
oe = OrdinalEncoder()
salaryClassification["occupation"] = oe.fit_transform(salaryClassification[[" occupation"]]).astype(int)
salaryClassification = salaryClassification.drop(' occupation', 1)

#convert 'relationship'n' from categorical to numeric
oe = OrdinalEncoder()
salaryClassification["relationship"] = oe.fit_transform(salaryClassification[[" relationship"]]).astype(int)
salaryClassification = salaryClassification.drop(' relationship', 1)

#convert 'race'n' from categorical to numeric
oe = OrdinalEncoder()
salaryClassification["race"] = oe.fit_transform(salaryClassification[[" race"]]).astype(int)
salaryClassification = salaryClassification.drop(' race', 1)

#convert 'sex'n' from categorical to numeric
oe = OrdinalEncoder()
salaryClassification["sex"] = oe.fit_transform(salaryClassification[[" sex"]]).astype(int)
salaryClassification = salaryClassification.drop(' sex', 1)

#join 2 columns(capital-gain and  capital-loss) in one
salaryClassification['capital-diff'] = salaryClassification[' capital-gain'] - salaryClassification[' capital-loss']
salaryClassification = salaryClassification.drop(' capital-gain', 1)
salaryClassification = salaryClassification.drop(' capital-loss', 1)

#fix column names
salaryClassification['hours-per-week'] = salaryClassification[' hours-per-week']
salaryClassification = salaryClassification.drop(' hours-per-week',1)


#convert 'native-country' from categorical to numeric
oe = OrdinalEncoder()
salaryClassification["native-county"] = oe.fit_transform(salaryClassification[[" native-country"]]).astype(int)
salaryClassification = salaryClassification.drop(' native-country', 1)


#convert salary-classification' from categorical to numeric
salaryClassification[' salary-classification'] = [x.replace(' <=50K', '0') for x in salaryClassification[' salary-classification']]
salaryClassification[' salary-classification'] = [x.replace(' >50K', '1') for x in salaryClassification[' salary-classification']]
salaryClassification[' salary-classification'] = salaryClassification[' salary-classification'].astype(int)

salaryClassification['salary-classification'] = salaryClassification[' salary-classification']
salaryClassification = salaryClassification.drop(' salary-classification',1)

X_train = salaryClassification.drop('salary-classification', 1)
y_train =  salaryClassification['salary-classification']

### Load test dataset 

In [14]:
testData = pd.read_csv('data/test.csv', sep = ';')

#treat missing values
testData = testData.replace(' ?', np.NaN)
testData[' workclass'] = testData[' workclass'].replace(np.NaN, 'Unknown')
testData[' occupation'] = testData[' occupation'].replace(np.NaN, 'Other')
testData[' native-country'] = testData[' native-country'].replace(np.NaN, 'Other')


#convert 'workclass' from categorical to numeric
oe = OrdinalEncoder()
testData["workclass"] = oe.fit_transform(testData[[" workclass"]]).astype(int)
testData = testData.drop(' workclass', 1)

#fix column name
testData['fnlwgt'] = testData[' fnlwgt']
testData = testData.drop(' fnlwgt',1)

#convert 'education' from categorical to numeric
oe = OrdinalEncoder()
testData["education"] = oe.fit_transform(testData[[" education"]]).astype(int)
testData = testData.drop(' education', 1)

#fix column name
testData['education-num'] = testData[' education-num']
testData = testData.drop(' education-num',1)

#convert 'education' from categorical to numeric
oe = OrdinalEncoder()
testData["marital-status"] = oe.fit_transform(testData[[" marital-status"]]).astype(int)
testData = testData.drop(' marital-status', 1)

#convert 'occupation'n' from categorical to numeric
oe = OrdinalEncoder()
testData["occupation"] = oe.fit_transform(testData[[" occupation"]]).astype(int)
testData = testData.drop(' occupation', 1)

#convert 'relationship'n' from categorical to numeric
oe = OrdinalEncoder()
testData["relationship"] = oe.fit_transform(testData[[" relationship"]]).astype(int)
testData = testData.drop(' relationship', 1)

#convert 'race'n' from categorical to numeric
oe = OrdinalEncoder()
testData["race"] = oe.fit_transform(testData[[" race"]]).astype(int)
testData = testData.drop(' race', 1)

#convert 'sex'n' from categorical to numeric
oe = OrdinalEncoder()
testData["sex"] = oe.fit_transform(testData[[" sex"]]).astype(int)
testData = testData.drop(' sex', 1)

#join 2 columns(capital-gain and  capital-loss) in one
testData['capital-diff'] = testData[' capital-gain'] - testData[' capital-loss']
testData = testData.drop(' capital-gain', 1)
testData = testData.drop(' capital-loss', 1)

#fix column name
testData['hours-per-week'] = testData[' hours-per-week']
testData = testData.drop(' hours-per-week',1)


#convert 'native-country' from categorical to numeric
oe = OrdinalEncoder()
testData["native-county"] = oe.fit_transform(testData[[" native-country"]]).astype(int)
testData = testData.drop(' native-country', 1)


#convert salary-classification' from categorical to numeric
testData[' salary-classification'] = [x.replace(' <=50K', '0') for x in testData[' salary-classification']]
testData[' salary-classification'] = [x.replace(' >50K', '1') for x in testData[' salary-classification']]
testData[' salary-classification'] = testData[' salary-classification'].astype(int)

testData['salary-classification'] = testData[' salary-classification']
testData = testData.drop(' salary-classification',1)

X_test = testData.drop('salary-classification', 1)
y_test =  testData['salary-classification']

#### Transformação dos dados

In [15]:
def robustScalingTest(train,test):
    scaler = RobustScaler()
    scaled_data = scaler.fit_transform( train )
    scaled_test = scaler.transform(test)
    return scaled_data, scaled_test;

def discretizeTest(X_train, X_test):
    ftd = ['age', 'hours-per-week', 'education-num', 'capital-diff']
    discretizer = KBinsDiscretizer(n_bins=3, encode='ordinal', strategy='uniform')
    X_train[ftd] = discretizer.fit_transform(X_train[ftd])
    X_test[ftd] = discretizer.transform(X_test[ftd])
    return X_train, X_test;

#####  Balanceamento

In [16]:
def smoteeenSampler(X_train, y_train):
    smote_enn = SMOTEENN(random_state=0)
    X_balanced, y_train = smote_enn.fit_sample(X_train, y_train)
    X_balanced, y_train = shuffle(X_balanced, y_train)
    return X_balanced, y_train;

def overSampler(X_train, y_train):
    ros = RandomOverSampler()
    X_balanced, y_train = ros.fit_sample(X_train, y_train)
    X_balanced, y_train = shuffle(X_balanced, y_train)
    return X_balanced, y_train;

def tomekSampler(X_train, y_train):
    cc = TomekLinks(sampling_strategy='majority')
    X_balanced, y_train = cc.fit_sample(X_train, y_train)
    X_balanced, y_train = shuffle(X_balanced, y_train)
    return X_balanced, y_train;

#### Feature Selection

In [7]:
def selectKBest_f_classif(X_train, y_train, X_test):
    kbest_selector_f_classif = SelectKBest(f_classif, k=7)
    selector = kbest_selector_f_classif.fit(X_train, y_train)
    #printFeatureSelection(selector, X_train)
    X_train_selected = kbest_selector_f_classif.transform(X_train)
    X_test_selected = kbest_selector_f_classif.transform(X_test)
    
    return X_train_selected, X_test_selected;

def selectKBest_chi2(X_train, y_train, X_test):
    kbest_selector_chi2 = SelectKBest(chi2, k=7)
    selector = kbest_selector_chi2.fit(X_train, y_train)
    #printFeatureSelection(selector, X_train)
    X_train_selected = kbest_selector_chi2.transform(X_train)
    X_test_selected = kbest_selector_chi2.transform(X_test)
    
    return X_train_selected, X_test_selected;

def selectPercentile_f_classif(X_train, y_train, X_test):
    percentile_selector_f_classif = SelectPercentile(f_classif, percentile=25)
    selector = percentile_selector_f_classif.fit(X_train, y_train)
    X_train_selected = percentile_selector_f_classif.transform(X_train)
    X_test_selected = percentile_selector_f_classif.transform(X_test)
    
    return X_train_selected, X_test_selected;

def selectPercentile_chi2(X_train, y_train, X_test):
    percentile_selector_chi2 = SelectPercentile(chi2, percentile=25)
    selector = percentile_selector_chi2.fit(X_train, y_train)
    X_train_selected = percentile_selector_chi2.transform(X_train)
    X_test_selected = percentile_selector_chi2.transform(X_test)
    
    return X_train_selected, X_test_selected;


def selectVarianceThreshold(X_train, y_train, X_test):
    varianceThreshold_selector = VarianceThreshold()
    selector = varianceThreshold_selector.fit(X_train, y_train)
    #printFeatureSelection(selector, X_train)
    X_train_selected = varianceThreshold_selector.transform(X_train)
    X_test_selected = varianceThreshold_selector.transform(X_test)
    
    return X_train_selected, X_test_selected;

def selectGenericUnivariateSelect(X_train, y_train, X_test):
    gus_selector = GenericUnivariateSelect(f_classif, 'k_best', param=14)
    selector = gus_selector.fit(X_train, y_train)
    X_train_selected = gus_selector.transform(X_train)
    X_test_selected = gus_selector.transform(X_test)
    
    return X_train_selected, X_test_selected;


def rfeLogReg(X_train, y_train, X_test):
    rfe_log_selector = RFE(LogisticRegression(), 12)
    selector = rfe_log_selector.fit(X_train, y_train)
    #printFeatureSelection(selector, X_balanced)
    X_train_selected = rfe_log_selector.transform(X_train)
    X_test_selected = rfe_log_selector.transform(X_test)
    
    return X_train_selected, X_test_selected;


def rfeSVC(X_train, y_train, X_test):
    rfe_svc_selector = RFE(SVC(), 12)
    selector = rfe_svc_selector.fit(X_train, y_train)
    #printFeatureSelection(selector, X_balanced)
    X_train_selected = rfe_svc_selector.transform(X_train)
    X_test_selected = rfe_svc_selector.transform(X_test)
    
    return X_train_selected, X_test_selected;


def sfmTree(X_train, y_train, X_test):
    tree_selector = ExtraTreesClassifier(n_estimators=50)
    selector = tree_selector.fit(X_train, y_train)
    sfm_Tree_selector = SelectFromModel(tree_selector, prefit=True)
    #printFeatureSelection(selector, X_balanced)
    X_train_selected = sfm_Tree_selector.transform(X_train)
    X_test_selected = sfm_Tree_selector.transform(X_test)
    
    return X_train_selected, X_test_selected;

#### Funções para grid search e aplicação das técnicas do pipeline

In [8]:
def gridSearch(model, param_grid, modelName, X_train, y_train, X_test, y_test):        
    clf = GridSearchCV(model, param_grid, refit=True, verbose=0)
    clf.fit(X_train,y_train)
    print(clf.best_params_)
    predicted = clf.predict(X_test)
    evaluateModel(modelName, y_test, predicted)
    return ;
    
    
def apllyGridSearchWithTransformation(model, transformer, param_grid, modelName):
    X_train = salaryClassification.drop('salary-classification', 1)
    y_train =  salaryClassification['salary-classification']
    X_test = testData.drop('salary-classification', 1)
    y_test =  testData['salary-classification']
    
    X_train, X_test = transformer(X_train, X_test)
    
    gridSearch(model, param_grid, modelName, X_train, y_train, X_test, y_test)
    return ;


def apllyGridSearchWithFSelect(model, transformer, selector, param_grid, modelName):
    X_train = salaryClassification.drop('salary-classification', 1)
    y_train =  salaryClassification['salary-classification']
    X_test = testData.drop('salary-classification', 1)
    y_test =  testData['salary-classification']
    
    X_train, X_test = transformer(X_train, X_test)
    X_train, X_test = selector(X_train, y_train, X_test)
    
    gridSearch(model, param_grid, modelName, X_train, y_train, X_test, y_test)
    return ;

def apllyGridSearchWithLoadBalancing(model, transformer, selector, balancer, param_grid, modelName):
    X_train = salaryClassification.drop('salary-classification', 1)
    y_train =  salaryClassification['salary-classification']
    X_test = testData.drop('salary-classification', 1)
    y_test =  testData['salary-classification']
    
    X_train, X_test = transformer(X_train, X_test)
    X_train, y_train = balancer(X_train, y_train)
    X_train, X_test = selector(X_train, y_train, X_test)

    gridSearch(model, param_grid, modelName, X_train, y_train, X_test, y_test)
    return ;


def evaluateModel(name, y_test, predicted):
    print("Accuracy: %0.3f || AUROC %0.3f || (Accuracy, Precision) 0:( %0.3f, %0.3f)  1:( %0.3f, %0.3f) ->" 
              % (accuracy_score(y_test,predicted), roc_auc_score(y_test, predicted),
                recall_score(y_test,predicted,pos_label=0), precision_score(y_test,predicted,pos_label=0),
                recall_score(y_test,predicted,pos_label=1), precision_score(y_test,predicted,pos_label=1)), name)
    return;

## Avaliação dos modelos 

### SVC

In [9]:
param_grid_svc = {
    'class_weight': ['balanced', None], 
    'C': [0.1,1, 10, 100], 
    'gamma': [1,0.1,0.01,0.001],
     'kernel': ['rbf']
} 

apllyGridSearchWithLoadBalancing(SVC(), discretizeTest, selectVarianceThreshold, tomekSampler, param_grid_svc, "SVC")

{'C': 0.1, 'gamma': 1, 'kernel': 'rbf'}
Accuracy: 0.824 || AUROC 0.698 || (Accuracy, Precision) 0:( 0.936, 0.849)  1:( 0.460, 0.689) -> SVC


### KNearest Neighbors

In [10]:
grid_params_knn = {
    'n_neighbors' : [3,5,7,11,13, 15, 17, 25, 30, 50],
    'weights' : ['uniform', 'distance'],
    'metric' : ['euclidean', 'manhattan']
}

apllyGridSearchWithLoadBalancing(KNeighborsClassifier(), discretizeTest, selectPercentile_chi2, overSampler, grid_params_knn, "knn")

apllyGridSearchWithLoadBalancing(KNeighborsClassifier(), discretizeTest, selectPercentile_chi2, tomekSampler, grid_params_knn, "knn")

apllyGridSearchWithLoadBalancing(KNeighborsClassifier(), discretizeTest, selectKBest_f_classif, tomekSampler, grid_params_knn, "knn")

{'metric': 'manhattan', 'n_neighbors': 3, 'weights': 'distance'}
Accuracy: 0.650 || AUROC 0.595 || (Accuracy, Precision) 0:( 0.700, 0.816)  1:( 0.490, 0.335) -> knn
{'metric': 'manhattan', 'n_neighbors': 50, 'weights': 'distance'}
Accuracy: 0.737 || AUROC 0.601 || (Accuracy, Precision) 0:( 0.859, 0.809)  1:( 0.343, 0.429) -> knn
{'metric': 'euclidean', 'n_neighbors': 50, 'weights': 'uniform'}
Accuracy: 0.822 || AUROC 0.708 || (Accuracy, Precision) 0:( 0.924, 0.855)  1:( 0.492, 0.666) -> knn


### Decision Tree

In [22]:
X_train = salaryClassification.drop('salary-classification', 1)
y_train =  salaryClassification['salary-classification']
X_test = testData.drop('salary-classification', 1)
y_test =  testData['salary-classification']

clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)
predicted = clf.predict(X_test)
print("Accuracy: %0.3f || AUROC %0.3f || (Accuracy, Precision) 0:( %0.3f, %0.3f)  1:( %0.3f, %0.3f) ->" 
              % (accuracy_score(y_test,predicted), roc_auc_score(y_test, predicted),
                recall_score(y_test,predicted,pos_label=0), precision_score(y_test,predicted,pos_label=0),
                recall_score(y_test,predicted,pos_label=1), precision_score(y_test,predicted,pos_label=1)), "DecisionTree")

Accuracy: 0.809 || AUROC 0.740 || (Accuracy, Precision) 0:( 0.871, 0.878)  1:( 0.608, 0.594) -> DecisionTree


### Multilayer Perceptron

In [11]:
grid_params_mlp = {
    'solver': ['adam'],
    'activation': ['identity', 'logistic','tanh'],
    'max_iter': [1000],
    'shuffle': [True],
    'hidden_layer_sizes': [100, 150, 200],
}

apllyGridSearchWithLoadBalancing(MLPClassifier(), discretizeTest, selectKBest_f_classif, tomekSampler, grid_params_mlp, "mlp")

{'activation': 'logistic', 'hidden_layer_sizes': 150, 'max_iter': 1000, 'shuffle': True, 'solver': 'adam'}
Accuracy: 0.822 || AUROC 0.714 || (Accuracy, Precision) 0:( 0.920, 0.858)  1:( 0.508, 0.661) -> mlp


### AdaBoost

In [9]:
grid_params_adaboost = {
     'n_estimators' : [10,20,30,50,100,200,1000],
      'learning_rate':[.001,0.01,.1],  
}

apllyGridSearchWithLoadBalancing(AdaBoostClassifier(), robustScalingTest, selectKBest_f_classif, tomekSampler, grid_params_adaboost, "AdaBoost")

{'learning_rate': 0.1, 'n_estimators': 1000}
Accuracy: 0.855 || AUROC 0.771 || (Accuracy, Precision) 0:( 0.931, 0.885)  1:( 0.611, 0.733) -> AdaBoost


### Random Forest

In [21]:
grid_params_randomforest = {
     'n_estimators' : [10,20,30,50,100,200,1000],
    'max_depth' : [1, 10, 20, None],
    'bootstrap': [True, False],
}

apllyGridSearchWithLoadBalancing(RandomForestClassifier(), robustScalingTest, selectKBest_f_classif, tomekSampler, grid_params_randomforest, "Random Forest")

{'bootstrap': True, 'max_depth': 10, 'n_estimators': 1000}
Accuracy: 0.858 || AUROC 0.768 || (Accuracy, Precision) 0:( 0.940, 0.882)  1:( 0.595, 0.754) -> Random Forest


### Voting Classifier

In [22]:
X_train = salaryClassification.drop('salary-classification', 1)
y_train =  salaryClassification['salary-classification']
X_test = testData.drop('salary-classification', 1)
y_test =  testData['salary-classification']

X_train, X_test = robustScalingTest(X_train, X_test)
X_train, X_test = selectKBest_f_classif(X_train, y_train, X_test)
X_train, y_train = tomekSampler(X_train, y_train)

clf1 = DecisionTreeClassifier()
clf2 = AdaBoostClassifier(learning_rate =0.1, n_estimators=1000)
clf3 = RandomForestClassifier(bootstrap= True, max_depth= 10, n_estimators= 1000)

eclf1 = VotingClassifier(estimators=[
('dtc', clf1), ('abc', clf2), ('rfc', clf3)], voting='hard')

eclf1 = eclf1.fit(X_train, y_train)

predicted = eclf1.predict(X_test)

print("Accuracy: %0.3f || AUROC %0.3f || (Accuracy, Precision) 0:( %0.3f, %0.3f)  1:( %0.3f, %0.3f) ->" 
              % (accuracy_score(y_test,predicted), roc_auc_score(y_test, predicted),
                recall_score(y_test,predicted,pos_label=0), precision_score(y_test,predicted,pos_label=0),
                recall_score(y_test,predicted,pos_label=1), precision_score(y_test,predicted,pos_label=1)), "voting classifier")

Accuracy: 0.860 || AUROC 0.760 || (Accuracy, Precision) 0:( 0.949, 0.877)  1:( 0.572, 0.775) -> voting classifier
